In [ ]:
import urllib
import json
import pandas as pd
from pandas import json_normalize
import time

In [ ]:
apikey = "example-api-key"
baseurl = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"


# Jay: define states as a list, not a tuple
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine",
    "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
    "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey",
    "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio",
    "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
    "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia",
    "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

years = list(range(2010, 2020)) # Add your desired years here
data = []

In [ ]:
## Connect to your MongoDB database
host = "mongodb.fsb.miamioh.edu"
port = 27017
user_name = "example-username"
pass_word = "example-pw"
db_name = "example"  # database name to authenticate

from pymongo import MongoClient
client = MongoClient(f'mongodb://{user_name}:{pass_word}@{host}:{port}', authSource="admin")
db= client[db_name]

# Jay: save articles to a testing collection
articles_col = db.stroke_articles3 #

In [ ]:
years[7:10]

[2017, 2018, 2019]

In [ ]:
# Jay: for the testing purpose, try two years and two states
for year in years[7:10]:
    print("Current year:", year)

    for state in states:
        print("Current state:", state)
        year_begin = str(year) + '0101'
        year_end = str(year) + '1231'

        #print("Current year:", year, "State:", state)

        # Define a list of keywords
        keywords_list = ["stroke", "health", "people"]

        # Combine the keywords using "AND" and create the query string
        keywords = " AND ".join(keywords_list)

        # Customize request parameter dictionary with the query string
        para_dict = {"api-key": apikey, "q": f"{state} {keywords}", "begin_date": year_begin, "end_date": year_end}


        # build the full url
        url2check = baseurl + urllib.parse.urlencode(para_dict)

        # send request and retrieve result
        result = urllib.request.urlopen(url2check).read()

        # parse result as a json object
        result_dict = json.loads(result)

        # get the hits value, i.e., number of the articles answering this query
        hits = result_dict['response']['meta']['hits']
        print("hits:", hits)

        # Jay: we don't need to save this hits number. But directly go to article extraction
        '''
        # append current data to the list
        data.append({"Year": year, "State": state, "Hits": hits})
        '''
        # Jay: hits_limit to page
        pages = hits // 10 + 1   # Jay: we can use floor division here, this operator will divide the first argument by the second and round the result down to the nearest whole number,
        print("pages:", pages)

        year_begin = str(year) + '0101'
        year_end = str(year) + '1231'

        # Jay: getting hits number also cost one request, so we need sleep 12 seconds here.
        time.sleep(15)

        for p in range(pages):
            print("Current page:", p)


            para_dict = {"api-key": apikey, "q": f"{state} {keywords}", "begin_date": year_begin, "end_date": year_end, "page": p}

            url2check = baseurl + urllib.parse.urlencode(para_dict)

            result = urllib.request.urlopen(url2check).read()

            result_dict = json.loads(result)

            # Jay: iterate through each article
            for doc in result_dict['response']['docs']:
                    doc["Year"] = year
                    doc["State"] = state
                    doc["Hits"] = hits
                #print(doc.get('web_url'))
                # Jay: we treat web_url as the id column for articles. check it to avoid duplicates

                    # Modify the _id to ensure uniqueness based on web URL and state
                    doc["_id"] = f"{doc['web_url']}_{state}"

                    # Check if the article is not already in the database
                    if articles_col.find_one({"_id": doc["_id"]}) is None:
                            articles_col.insert_one(doc)

                #if articles_col.find_one({"web_url": doc.get('web_url')}) is None: # only insert those articles not in database
                    #articles_col.insert_one(doc)


            time.sleep(15)

Current year: 2017
Current state: Alabama
hits: 9
pages: 1
Current page: 0
Current state: Alaska
hits: 4
pages: 1
Current page: 0
Current state: Arizona
hits: 17
pages: 2
Current page: 0
Current page: 1
Current state: Arkansas
hits: 2
pages: 1
Current page: 0
Current state: California
hits: 35
pages: 4
Current page: 0
Current page: 1
Current page: 2
Current page: 3
Current state: Colorado
hits: 2
pages: 1
Current page: 0
Current state: Connecticut
hits: 15
pages: 2
Current page: 0
Current page: 1
Current state: Delaware
hits: 4
pages: 1
Current page: 0
Current state: Florida
hits: 27
pages: 3
Current page: 0
Current page: 1
Current page: 2
Current state: Georgia
hits: 10
pages: 2
Current page: 0
Current page: 1
Current state: Hawaii
hits: 3
pages: 1
Current page: 0
Current state: Idaho
hits: 1
pages: 1
Current page: 0
Current state: Illinois
hits: 7
pages: 1
Current page: 0
Current state: Indiana
hits: 8
pages: 1
Current page: 0
Current state: Iowa
hits: 3
pages: 1
Current page: 0
Curr

In [ ]:
articles_col

Collection(Database(MongoClient(host=['mongodb.fsb.miamioh.edu:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin'), 'voat'), 'stroke_articles3')